In [ ]:
# @title Mounting Drive
#mounting google drive
from google.colab import drive
drive.mount('/gdrive')

**#Import Libraries**

In [1]:
!git remote add origin https://github.com/MAIA-Group-Project/multi_disciplinary_project.git


fatal: not a git repository (or any of the parent directories): .git


In [2]:
!git add .
!git commit -m "Your commit message"

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [3]:
!git push -u origin master


fatal: not a git repository (or any of the parent directories): .git


In [27]:
# @title Libraries
import os
import zipfile
import json
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
# @title Extract the dataset

# Define the path to dataset zip file in Google Drive
zip_file_path = '/content/drive/My Drive/X-ray panoramic radiography image dataset.zip'

# Define the directory where to extract the  dataset
extracted_dir_path = '/content/drive/My Drive/dataset'

# Extract the dataset
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

# List the contents of the extracted directory
extracted_files = os.listdir(extracted_dir_path)
print("Dataset extracted successfully. Contents of the directory:")
print(extracted_files)




In [28]:
# @title Dataset directory and sub folders
# Define the directory where the dataset is extracted
dataset_dir = '/content/drive/My Drive/dataset/X-ray panoramic radiography image dataset'

# Define paths to subfolders
radiographs_dir = os.path.join(dataset_dir, 'Radiographs')
segmentation_dir = os.path.join(dataset_dir, 'Segmentation/teeth_mask')
masks_dir = os.path.join(dataset_dir, 'Expert/mask')
expert_dir= os.path.join(dataset_dir, 'Expert')
expert_json_path = os.path.join(expert_dir, 'expert.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [29]:
# Function to load images from a directory
def load_images_from_directory(directory):
    images = []
    for filename in os.listdir(directory):
        image_path = os.path.join(directory, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            images.append(image)
    return images

In [30]:
# Function to display sample images
def display_sample_images(directory, title):
    plt.figure(figsize=(15, 6))
    plt.suptitle(title, fontsize=16)
    for i, filename in enumerate(os.listdir(directory)[:5]):
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)
        plt.subplot(2, 5, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(filename)
    plt.show()


In [ ]:
# @title Display Sample Images
# Display sample radiographs
display_sample_images(radiographs_dir, 'Sample Radiographs')

# Display sample radiographs segmentations
display_sample_images(segmentation_dir, 'Sample Segmentation')

# Display sample abnormality location masks
display_sample_images(masks_dir, 'Sample Masks')

In [ ]:
# @title Extract JSON files
#
# Check if the 'expert.json' file exists
if os.path.exists(expert_json_path):
    # Open and read the contents of the 'expert.json' file
    with open(expert_json_path, 'r') as f:
        expert_data = json.load(f)

        # Loop through the first 10 entries in the JSON file
        for i, entry in enumerate(expert_data[:20]):
            print("Entry", i+1)
            print("-------------------")

            # Access label information
            labels = entry.get('Label', {}).get('objects', [])

            # Loop through each label object
            for label_obj in labels:
                title = label_obj.get('title')
                value = label_obj.get('value')
                if title and value:  # Check if title and value exist
                    print("Label:", title)
                    print("Value:", value)

                    # Access classifications if available
                    classifications = label_obj.get('classifications', [])
                    if classifications and isinstance(classifications, list):  # Check if classifications exist and is a list
                        print("Classifications:")
                        for classification in classifications:
                            if isinstance(classification, dict):  # Check if classification is a dictionary
                                class_title = classification.get('title')
                                class_value = classification.get('answer', {}).get('title')
                                if class_title and class_value:  # Check if class_title and class_value exist
                                    print("- {}: {}".format(class_title, class_value))

                    print()

            # Access description and external ID
            description = entry.get('Description')
            external_id = entry.get('External ID')
            if description:  # Check if description exists
                print("Description:", description)
            if external_id:  # Check if external ID exists
                print("External ID:", external_id)

            print("-------------------\n")
else:
    print("The 'expert.json' file does not exist in the 'Expert' folder.")

**#Apply Preprocessing Techniques:**
   #Contrast Enhancement:

In [32]:
# Function to apply histogram equalization
def equalize_histogram(images):
    equalized_images = [cv2.equalizeHist(img) for img in images]
    return equalized_images

In [33]:
# Apply histogram equalization
radiograph_images = load_images_from_directory(radiographs_dir)
equalized_images = [cv2.equalizeHist(img) for img in radiograph_images]

In [35]:

def plot_histogram(images):
    plt.figure(figsize=(15, 5))
    for i in range(5):  # Plot 5 sample images
        plt.subplot(1, 5, i+1)
        plt.hist(images[i].ravel(), bins=256, range=[0, 256])
        plt.title('Image ' + str(i+1) + ' Histogram')
    plt.show()

In [ ]:
# Plot histogram of original images
plot_histogram(radiograph_images)

# Plot histogram of equalized images
plot_histogram(equalized_images)

#CLAHE

In [ ]:
#Contrast Limited Adaptive Histogram Equalization
output_dir = "/content/drive/My Drive/dataset/CLAHE"
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
# Parameters
clip_limit = 2.0
tile_grid_size = (8, 8)

# Iterate through images in the input directory
for filename in os.listdir(radiographs_dir):
    # Read the image
    img = cv2.imread(os.path.join(radiographs_dir, filename), cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue  # Skip if the image cannot be read
    #Normalize images using min-max normalization
    normalized_image=cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clip_limit, tile_grid_size)
    enhanced_img = clahe.apply(normalized_image)

    # Save the enhanced image in the output directory
    cv2.imwrite(os.path.join(output_dir, filename), enhanced_img)


#**Spatial Filtering**

In [ ]:
# @title Default title text
import cv2
import os

# Input and output directories
input_dir = "/content/drive/My Drive/dataset/CLAHE"
gaussian_dir= "/content/drive/My Drive/dataset/Denoised_Gausian"
median_dir = "/content/drive/My Drive/dataset/Denoised_Median"
bilateral_dir = "/content/drive/My Drive/dataset/Denoised_Bilateral"
output_dir = "/content/drive/My Drive/dataset/Denoised_NLM"

# Create output directory if it doesn't exist
os.makedirs(gaussian_dir, exist_ok=True)
os.makedirs(median_dir, exist_ok=True)
os.makedirs(bilateral_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# Parameters for non-local means filtering
h_value = 10
search_window_size = 21
# Iterate through images in the input directory
for filename in os.listdir(input_dir):
    # Read the CLAHE-enhanced image
    img = cv2.imread(os.path.join(input_dir, filename), cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue  # Skip if the image cannot be read
    # Gaussian Filtering
    gaussian_img = cv2.GaussianBlur(img, (5, 5), 0)
    cv2.imwrite(os.path.join(gaussian_dir, "gaussian_" + filename), gaussian_img)

    # Median Filtering
    median_img = cv2.medianBlur(img, 5)
    cv2.imwrite(os.path.join(median_dir , "median_" + filename), median_img)

    # Bilateral Filtering
    bilateral_img = cv2.bilateralFilter(img, 9, 75, 75)
    cv2.imwrite(os.path.join(bilateral_dir, "bilateral_" + filename), bilateral_img)

    # Apply non-local means filtering
    denoised_img = cv2.fastNlMeansDenoising(img, None, h=h_value, templateWindowSize=search_window_size, searchWindowSize=search_window_size)
    cv2.imwrite(os.path.join(output_dir, filename), denoised_img)


In [38]:
import cv2
import os

# Define directories
denoised_radiograph = "/content/drive/My Drive/dataset/Denoised_NLM"
opening_dir = "/content/drive/My Drive/dataset/Opening"
top_hat_dir = "/content/drive/My Drive/dataset/Top_Hat"

# Create directories if they don't exist
os.makedirs(opening_dir, exist_ok=True)
os.makedirs(top_hat_dir, exist_ok=True)

# Define kernel size
kernel_size = (5, 5)

# Iterate over each image in the folder
for filename in os.listdir(denoised_radiograph):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Read the image
        image_path = os.path.join(denoised_radiograph, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Apply opening operation
        opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size))

        # Apply top hat operation
        top_hat = cv2.morphologyEx(image, cv2.MORPH_TOPHAT, cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size))

        # Write original, opening, and top hat images
        cv2.imwrite(os.path.join(opening_dir, filename), opening)
        cv2.imwrite(os.path.join(top_hat_dir, filename), top_hat)


**#Morphological Operations**

In [ ]:
 #apply region growing based segmentation
    seed_poin=(100,100)
    region_grown=cv2.floodFill(top_hat, None, seed_poit, 255, flags=cv2.FLOODFILL_MASK_ONLY)
    #disply the result of region growing


    #apply adaptive thresholding
    thresholded_image=cv2.adaptiveThreshold(top_hat,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)

    #display the result of thresholding



    #find contours

    countours=cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #filter out contours based on area
    min_area=100
    filtered_contours=[cnt for cnt in countours if cv2.contourArea(cnt)>min_area]
    #drawu contours
    contour_image=np.zeros_like(image)
    cv2.drawContours(contour_image, filtered_contours, -1,(255,255,255), thickness=cv2.FILLED)



    #display the result of contour


#save the processed image
processed_image_path=os.path.join(denoised_radiograph, "processed"+filename)
cv2.imwrite(processed_image_path, contour_image)
processed_images.append(processed_image_path)


# randomly selecta sample iamges to display
num_samples=min(5, len(processed_images))
sample_images=random.sample(processed_images, num_samples)

#display the sample images
for im_path in sample_images:
  img=cv2.imread(im_path)
  cv2.imshow("segmented", img)
  ecv2.waitkey(0)
  cv2.destroyAllWindow()